In [70]:
import pandas as pd
movies_data = pd.read_table("train.tsv")
movies_data.shape
movies_data["Sentiment"].unique()

array([1, 2, 3, 4, 0], dtype=int64)

In [71]:
movie_test = pd.read_table("test.tsv")
movie_test.to_csv("test.csv", index= False)
movie_test.shape

(66292, 3)

In [72]:
cols = ["Phrase", "Sentiment"]
movies_data_new =movies_data[cols] 

In [73]:
movies_data_new.to_csv("train.csv", index= False)

In [74]:
import csv
with open("test.csv", 'r') as file: 
    test = list(csv.reader(file))

print(len(test))

66293


In [75]:
import csv
with open("train.csv", 'r') as file: 
    reviews = list(csv.reader(file))

In [76]:
reviews

[['Phrase', 'Sentiment'],
 ['A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .',
  '1'],
 ['A series of escapades demonstrating the adage that what is good for the goose',
  '2'],
 ['A series', '2'],
 ['A', '2'],
 ['series', '2'],
 ['of escapades demonstrating the adage that what is good for the goose', '2'],
 ['of', '2'],
 ['escapades demonstrating the adage that what is good for the goose', '2'],
 ['escapades', '2'],
 ['demonstrating the adage that what is good for the goose', '2'],
 ['demonstrating the adage', '2'],
 ['demonstrating', '2'],
 ['the adage', '2'],
 ['the', '2'],
 ['adage', '2'],
 ['that what is good for the goose', '2'],
 ['that', '2'],
 ['what is good for the goose', '2'],
 ['what', '2'],
 ['is good for the goose', '2'],
 ['is', '2'],
 ['good for the goose', '3'],
 ['good', '3'],
 ['for the goose', '2'],
 ['for', '2'],
 ['the goose

In [77]:
def get_text(reviews, score):
    # Join together the text in the reviews for a particular tone.
    #We lowercase to avoid "Not" and "not" being seen as different words, for example.
    return " ".join([r[0].lower() for r in reviews if r[1] == str(score)])

In [78]:
negative_text = get_text(reviews, 0)
somewhat_negative_text = get_text(reviews, 1)
neutral_text = get_text(reviews, 2)
somewhat_positive_text = get_text(reviews, 3)
positive_text = get_text(reviews, 4)

In [79]:
positive_text

"this quiet , introspective and entertaining independent is worth seeking . quiet , introspective and entertaining independent entertaining is worth seeking a positively thrilling combination of ethnography and all the intrigue , betrayal , deceit and murder of a shakespearean tragedy or a juicy soap opera a positively thrilling combination of ethnography and all the intrigue , betrayal , deceit and murder thrilling a comedy-drama of nearly epic proportions rooted in a sincere performance by the title character undergoing midlife crisis . nearly epic rooted in a sincere performance by the title character undergoing midlife crisis . in a sincere performance a sincere performance sincere performance recommend snow dogs high hilarity the performances are an absolute joy . are an absolute joy . an absolute joy joy extravagant better this one is a sweet and modest and ultimately winning story . is a sweet and modest and ultimately winning story . a sweet and modest sweet ultimately winning 

In [80]:
from collections import Counter
import re
def count_text(text):
    # Split text into words based on whitespace.  Simple but effective.
    words = re.split("\s+", text)
    #Count up the occurence of each word.
    return Counter(words)

In [81]:
negative_counts = count_text(negative_text)
somewhat_negative_counts = count_text(somewhat_negative_text)
neutral_counts = count_text(neutral_text)
somewhat_positive_counts = count_text(somewhat_positive_text)
positive_counts = count_text(positive_text)

total_counts = negative_counts+somewhat_negative_counts+neutral_counts+somewhat_positive_counts+positive_counts
print(len(total_counts))

16531


Here the most important column which we will decide the sentiment is the Phrase. Since contains the text.

We Will split each phrase on space and store the words in a list and the corresponding score will be the score of the Sentiment. For all the common words we will use the score which occurs maximum times.

In [82]:
def get_text(reviews,score):
    # Join together the text in the reviews for a particular tone.
    #We lowercase to avoid "Not" and "not" being seen as different words, for example.
    return " ".join([r[0].lower() for r in reviews if r[1] == str(score)])

In [83]:
import re
from collections import Counter
sentiments = {}
def collectwords(review): 
    words = re.split("\s+", review[2].lower())
        #Count up the occurence of each word.
        #words = words.toLower()
    return Counter(words)

#Sentiment wise list - 
words_list_sentiment1 = movies_data.apply(collectwords, axis = 1)
words_list_sentiment1[0]

Counter({',': 1,
         '.': 1,
         'a': 2,
         'adage': 1,
         'also': 1,
         'amounts': 1,
         'amuses': 1,
         'but': 1,
         'demonstrating': 1,
         'escapades': 1,
         'for': 2,
         'gander': 1,
         'good': 2,
         'goose': 1,
         'is': 2,
         'much': 1,
         'none': 1,
         'occasionally': 1,
         'of': 4,
         'series': 1,
         'some': 1,
         'story': 1,
         'that': 1,
         'the': 3,
         'to': 1,
         'what': 1,
         'which': 2})

In [84]:
def get_y_count(score):
    # Compute the count of each classification occurring in the data.
    return len([r for r in reviews if r[1] == str(score)])
    #We need these counts to use for smoothing when computing the prediction.

negative_review_count = get_y_count(0) 
somewhat_negative_review_count = get_y_count(1)
neutral_review_count = get_y_count(2)
somewhat_positive_review_count = get_y_count(3)
positive_review_count = get_y_count(4)

In [85]:
prob_negative_review = negative_review_count / len(reviews) 
prob_somewhat_negative_review = somewhat_negative_review_count / len(reviews)
prob_neutral_review = neutral_review_count / len(reviews)
prob_somewhat_positive_review = somewhat_positive_review_count / len(reviews)
prob_positive_review = positive_review_count / len(reviews)

In [ ]:
def make_class_prediction(text, counts, class_prob, class_count): 
    prediction = 1 
    text_counts = Counter(re.split("\s+", text)) 
    for word in text_counts:
        # For every word in the text, we get the number of times that word occured in the reviews for a given class, add 1 to smooth the value, and divide by the total number of words in the class (plus the class_count to also smooth the denominator).
        #Smoothing ensures that we don't multiply the prediction by 0 if the word didn't exist in the training data.
        #We also smooth the denominator counts to keep things even.
        prediction *=  text_counts.get(word) * ((counts.get(word, 0) + 1) / (sum(counts.values()) + class_count))
        #Now we multiply by the probability of the class existing in the documents.
    return prediction * class_prob

In [ ]:
import csv





def make_decision(text, make_class_prediction):
    # Compute the negative and positive probabilities.
    negative_prediction = make_class_prediction(text, negative_counts, prob_negative_review, negative_review_count)
    somewhat_negative_prediction = make_class_prediction(text, somewhat_negative_counts, prob_somewhat_negative_review, somewhat_negative_review_count)
    neutral_prediction = make_class_prediction(text, neutral_counts, prob_neutral_review, neutral_review_count)
    somewhat_positive_prediction = make_class_prediction(text, somewhat_positive_counts, prob_somewhat_positive_review, somewhat_positive_review_count)
    positive_prediction = make_class_prediction(text, positive_counts, prob_positive_review, positive_review_count)

    predictions = [negative_prediction,somewhat_negative_prediction,neutral_prediction,somewhat_positive_prediction,
                   positive_prediction]
    return predictions.index(max(predictions))


with open("test.csv", 'r') as file:
    test = list(csv.reader(file))


predictions = [make_decision(r[2], make_class_prediction) for r in test]

In [ ]:
predictions

In [ ]:
del predictions[0]
print(len(predictions))
print(movie_test.shape)


In [ ]:
result = pd.DataFrame({
    "PhraseId": movie_test["PhraseId"],
    "Sentiment": predictions
    }
)
result.to_csv("first_submission.csv", index= False)

# Second Submission - MultinomialNB

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd

movies_train = pd.read_table("train.tsv")
movies_test = pd.read_table("test.tsv")

#instantiate classifier and vectorizer
clf=MultinomialNB(alpha=.01)
vectorizer =TfidfVectorizer(min_df=1,ngram_range=(1,2))

#Apply vectorizer to training data
traindata= movies_train["Phrase"]
X_train=vectorizer.fit_transform(traindata)
print(X_train.shape)
#Label Ids
y_train=movies_train["Sentiment"]

#Train classifier
clf.fit(X_train, y_train)
test_data = movies_test["Phrase"]

x_test = vectorizer.transform(test_data)

clf.predict(x_test)

TypeError: unorderable types: tuple() < int()

In [52]:
predictions = clf.predict(x_test)

In [53]:
predictions

array([3, 3, 2, ..., 1, 1, 2], dtype=int64)

In [54]:
resultsecond = pd.DataFrame({
    "PhraseId": movies_test["PhraseId"],
    "Sentiment": predictions
    }
)
resultsecond.to_csv("second_submission_multinomialnb_sklearn.csv", index= False)

# Third Submission - BernoulliNB

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
import pandas as pd

movies_train = pd.read_table("train.tsv")
movies_test = pd.read_table("test.tsv")

#instantiate classifier and vectorizer
clf=BernoulliNB(alpha=1.0)
vectorizer =TfidfVectorizer(min_df=1,ngram_range=(1,2))

#Apply vectorizer to training data
traindata= movies_train["Phrase"]
X_train=vectorizer.fit_transform(traindata)
print(X_train.shape)
#Label Ids
y_train=movies_train["Sentiment"]

#Train classifier
clf.fit(X_train, y_train)
test_data = movies_test["Phrase"]

x_test = vectorizer.transform(test_data)

predictions = clf.predict(x_test)

(156060, 94644)


In [45]:
resultsthird = pd.DataFrame({
    "PhraseId": movies_test["PhraseId"],
    "Sentiment": predictions
    }
)
resultsthird.to_csv("third_submission_bernoullinb_sklearn.csv", index= False)

# Improving the score of Naive Bayes


1. The Naive Bayes model makes strong assumptions about the data: it assumes that
words in a document are independent. This assumption is clearly violated in natural
language text: there are various types of dependences between words induced by the
syntactic, semantic, pragmatic and conversational structure of a tex

2.There are two possible approaches to improve the performance: (i)
modify the data, (ii) modify the classifier (or the probabilistic model).

3. The most apparent difference between these models is that the Poisson model and
the multinomial model use word occurrence frequencies, while the binary independence
model uses binary word occurrences. In this paper we consider the multinomial Naive
Bayes model because it is generally superior to the binary independence model for text
classification

4.The multinomial Naive Bayes model treats each occurrence of a word in a document
independently of any other occurrence of the same word. In reality, however, multiple
occurrences of the same word in a document are not independent. When a word occurs
once, it is likely to occur again, i.e. the probability of the second occurrence is much
higher than that of the first occurrence. This is called burstiness. The multinomial
model does not account for this phenomenon. This results in a large underestimation of
the probability of documents with multiple occurrences of the same word.

5. All non-alphanumeric
characters are removed, and letters are converted to lower case. In addition, a 100 word
stoplist is applied to the newsgroups articles and all numbers are mapped to a special
token. F



# Fouth Submission - MultinomialNB with tuning parameters GridSearchCV

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
#from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import GridSearchCV
import pandas as pd

movies_train = pd.read_table("train.tsv")
movies_test = pd.read_table("test.tsv")

#instantiate classifier and vectorizer
clf=MultinomialNB()
vectorizer =TfidfVectorizer(min_df=1, ngram_range=(1,2))
#bnl = BernoulliNB()
#Apply vectorizer to training data
traindata= movies_train["Phrase"]
X_train=vectorizer.fit_transform(traindata)
print(X_train.shape)
#Label Ids
y_train=movies_train["Sentiment"]

#Train classifier
clf.fit(X_train, y_train)

clf.get_params()

(156060, 94644)


{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [34]:
param_grid = {
    "alpha" :[.0001,.001, .01, 1.0],
    
    }
gs = GridSearchCV(clf,param_grid)
gs.fit(X_train, y_train)
print(gs.best_estimator_)
test_data = movies_test["Phrase"]

x_test = vectorizer.transform(test_data)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


In [43]:
predictions = gs.predict(x_test)
print(gs.best_estimator_)
print(gs.best_score_)
print(gs.best_params_ )
print(gs.best_index_)
print(gs.scorer_)
print(gs.n_splits_)
print(gs.cv_results_)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
0.54784698193
{'alpha': 1.0}
3
<function _passthrough_scorer at 0x0C33C540>
3
{'std_fit_time': array([ 0.01040365,  0.00249466,  0.01537038,  0.00235725]), 'split0_test_score': array([ 0.47910499,  0.47735573,  0.47666372,  0.55778325]), 'split2_test_score': array([ 0.45074782,  0.4500942 ,  0.44944058,  0.54450383]), 'split1_train_score': array([ 0.76236063,  0.76111111,  0.75928489,  0.69107074]), 'param_alpha': masked_array(data = [0.0001 0.001 0.01 1.0],
             mask = [False False False False],
       fill_value = ?)
, 'split2_train_score': array([ 0.76076008,  0.75947214,  0.75844371,  0.68945234]), 'mean_test_score': array([ 0.45729847,  0.45613226,  0.45553633,  0.54784698]), 'split0_train_score': array([ 0.75527211,  0.75400334,  0.75296526,  0.69006517]), 'std_score_time': array([ 0.00094291,  0.00047148,  0.        ,  0.00124726]), 'std_test_score': array([ 0.01582431,  0.01546543,  0.01537879,  0.00715041]), 'r

In [32]:
predictions

array([3, 3, 2, ..., 2, 2, 2], dtype=int64)

In [33]:
resultsfourth = pd.DataFrame({
    "PhraseId": movies_test["PhraseId"],
    "Sentiment": predictions
    }
)
resultsfourth.to_csv("fourth_submission_multinomialnb_tuningparameter_sklearn.csv", index= False)

# Fifth Submission - MultinomialNB with tuning parameters GridSearchCV

In [57]:
from sklearn.grid_search import RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
#from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import GridSearchCV
import pandas as pd

movies_train = pd.read_table("train.tsv")
movies_test = pd.read_table("test.tsv")

#instantiate classifier and vectorizer
clf=MultinomialNB()
vectorizer =TfidfVectorizer(min_df=1, ngram_range=(1,2))
#bnl = BernoulliNB()
#Apply vectorizer to training data
traindata= movies_train["Phrase"]
X_train=vectorizer.fit_transform(traindata)
print(X_train.shape)
#Label Ids
y_train=movies_train["Sentiment"]

#Train classifier
clf.fit(X_train, y_train)

param_grid = {
    "alpha" :[.0001,.001, .01, 1.0],
    
    }
gs = RandomizedSearchCV(clf,param_distributions=param_grid, n_iter=2)
gs.fit(X_train, y_train)
print(gs.best_estimator_)
test_data = movies_test["Phrase"]

x_test = vectorizer.transform(test_data)


(156060, 94644)
MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)


In [58]:
predictions = gs.predict(x_test)
resultsfifth = pd.DataFrame({
    "PhraseId": movies_test["PhraseId"],
    "Sentiment": predictions
    }
)
resultsfifth.to_csv("fifth_submission_multinomialnb_tuningparameter_randomsearhc_sklearn.csv", index= False)